# Pre-processing des données de TME

## 0. Import des données

Import des librairies **Numpy**, **Matplotlib** et **Pandas**

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set_style('ticks')

Import des données **pdv** sur les points de ventes, **titres_best** sur les titres en vente, **ventes_process** sur les transactions

In [4]:
df_pdv = pd.read_csv('CSV/pdv.csv',
                     sep=',',decimal=',',error_bad_lines=False,
                     index_col=0,dtype='unicode')
df_titres = pd.read_csv("CSV/titres_best.csv",
                        sep=',',error_bad_lines=False,index_col=0,
                        dtype='unicode')
df_ventes = pd.read_csv("CSV/ventes_process.csv",
                        sep=',',error_bad_lines=False,index_col=0,
                        dtype='unicode')

## 1. Traitement de pdv.csv

### Noms des colonnes

In [5]:
df_pdv.columns=["numero_pdv","departement","lineaire_sol",
                "lineaire_developpe","region","code_postal","localite",
                "lundi_ouv_am","lundi_ferm_am","lundi_ouv_pm","lundi_ferm_pm",
                "mardi_ouv_am","mardi_ferm_am","mardi_ouv_pm","mardi_ferm_pm",
                "mercredi_ouv_am","mercredi_ferm_am","mercredi_ouv_pm","mercredi_ferm_pm",
                "jeudi_ouv_am","jeudi_ferm_am","jeudi_ouv_pm","jeudi_ferm_pm",
                "vendredi_ouv_am","vendredi_ferm_am","vendredi_ouv_pm","vendredi_ferm_pm",
                "samedi_ouv_am","samedi_ferm_am","samedi_ouv_pm","samedi_ferm_pm",
                "dimanche_ouv_am","dimanche_ferm_am","dimanche_ouv_pm","dimanche_ferm_pm",
                "jourferie_ouv_am","jourferie_ferm_am","jourferie_ouv_pm","jourferie_ferm_pm",
                "activite_alimentation","activite_none","activite_bar_cafe",
                "activite_bazar_bricolage","activite_cadeaux_jouets",
                "activite_carterie","activite_confiserie",
                "activite_francaisedesjeux","activite_hotel_camping",
                "activite_librairie","activite_libreservice","activite_loto",
                "activite_pmu","activite_papeterie","activite_photo_video",
                "activite_relaiscolis","activite_souvenir","activite_stationservice",
                "activite_tabac","env_administration_publique",
                "env_arret_transport_commun","env_autoroute","env_campagne_rural",
                "env_centrecommercial","env_commerce_centreville",
                "env_commerce_peripherie","env_ecole_primaire","env_ecole_secondaire_sup",
                "env_gare_metro","env_hopital","env_hotel_camping","env_immeubles",
                "env_lieu_culte","env_lieu_sportif","env_lieu_touristique",
                "env_route_nationale_departementale","env_station_balnéaire",
                "env_station_ski","env_ZAC_ZI_ZAE","env_zone_pavillonnaire",
                "ouverture_samedi","ouverture_dimanche","ouverture_jourferie"]

### Numéros de point de vente
- nump_pdv est encodé en string et il existe des num_pdv ayant des espaces

In [6]:
df_pdv.shape

(24736, 82)

In [7]:
def test_if_to_delete_num(x) :
    if ' ' in x :
        return True
    if 'Inco' in x :
        return True
    return False

nb_pdv_espace = df_pdv.numero_pdv.apply(test_if_to_delete_num).sum()

print("Nb de point de vente contenant des espaces :", nb_pdv_espace)
print("Les lignes correspondantes seront retirées du dataframe")
df_pdv = df_pdv[~df_pdv.numero_pdv.apply(test_if_to_delete_num)]
df_pdv.shape

Nb de point de vente contenant des espaces : 34
Les lignes correspondantes seront retirées du dataframe


(24702, 82)

#### Convertir en int64

In [8]:
df_pdv['numero_pdv'] = df_pdv['numero_pdv'].astype('int64')

#### Doublons

In [9]:
print(df_pdv[df_pdv.numero_pdv.duplicated()].shape[0], 'pdv correspondent à des doublons')
print("On réfléchira plus tard à comment les gérer")

1884 pdv correspondent à des doublons
On réfléchira plus tard à comment les gérer


### Localisation

In [10]:
df_pdv.localite.isna().sum()
#df_pdv.region.isna().sum()
#df_pdv.departement.isna().sum()

0

In [11]:
strings = ["departement", "region", "localite"]

for col in strings:
    df_pdv[col] = df_pdv[col].astype(str)

### Linéaire au sol et linéaire developpé

In [12]:
df_pdv["lineaire_sol"] = df_pdv["lineaire_sol"].str.replace(',','.').astype(float)
df_pdv["lineaire_developpe"] = df_pdv["lineaire_developpe"].str.replace(',','.').astype(float)

In [13]:
print('nb lineare_sol NULL :', df_pdv.lineaire_sol.isnull().sum())
print('nb lineare_sol ZEROS :', (df_pdv.lineaire_sol == 0.0).sum())
print('nb lineaire_developpe ZEROS :', (df_pdv.lineaire_developpe == 0.0).sum())

nb lineare_sol NULL : 15
nb lineare_sol ZEROS : 653
nb lineaire_developpe ZEROS : 692


- Les valeurs NULL et nulles seront remplacées par la moyenne par département.

In [14]:
tmp = df_pdv[(df_pdv.lineaire_sol.isnull() | df_pdv.lineaire_sol == 0.0)][['numero_pdv', 'departement','lineaire_sol', 'lineaire_developpe']]
tmp.head()

,numero_pdv,departement,lineaire_sol,lineaire_developpe
495,42279,03 - ALLIER,0.0,5.6
669,87696,03 - ALLIER,0.0,0.0
1171,83387,06 - ALPES MARITIMES,0.0,0.0
1549,9065,09 - ARIEGE,0.0,20.0
1851,34852,11 - AUDE,0.0,0.0


In [15]:
for depart in df_pdv.departement.unique():
    condition_sol = ((df_pdv.departement == depart) &\
                    (df_pdv.lineaire_sol.isnull() | df_pdv.lineaire_sol == 0.0))
    condition_dev = (df_pdv.departement == depart) &\
                    (df_pdv.lineaire_developpe.isnull() | df_pdv.lineaire_developpe == 0.0)
    df_pdv.loc[condition_sol, 'lineaire_sol'] = np.nan
    df_pdv.loc[condition_dev, 'lineaire_developpe'] = np.nan
    
    moyenne_par_dep = df_pdv.loc[df_pdv.departement == depart].lineaire_sol.mean()
    nb_etag_moyen = (df_pdv[df_pdv.departement == depart].lineaire_developpe/\
                     df_pdv[df_pdv.departement == depart].lineaire_sol).mean()   
    
    df_pdv.loc[condition_sol, 'lineaire_sol'] = moyenne_par_dep
    df_pdv.loc[condition_dev, 'lineaire_developpe'] = nb_etag_moyen * df_pdv.loc[condition_sol, 'lineaire_sol']

In [16]:
df_pdv[['numero_pdv', 'departement', 'lineaire_sol', 'lineaire_developpe']].loc[tmp.index[:5]]

,numero_pdv,departement,lineaire_sol,lineaire_developpe
495,42279,03 - ALLIER,7.899167,5.600000
669,87696,03 - ALLIER,7.899167,72.549980
1171,83387,06 - ALPES MARITIMES,9.918065,92.078453
1549,9065,09 - ARIEGE,9.007229,20.000000
1851,34852,11 - AUDE,8.788884,80.325417


### Code postal 

* Codes nuls ou inconnus

In [17]:
print(df_pdv.code_postal.apply(test_if_to_delete_num).sum(), "codes postaux sont inconnus")
print("Les lignes associées seront supprimées")

5 codes postaux sont inconnus
Les lignes associées seront supprimées


In [18]:
df_pdv = df_pdv[~df_pdv.code_postal.apply(test_if_to_delete_num)]
df_pdv.shape

(24697, 82)

* Codes incorrects : on ajoute un 0 devant ceux à 4 chiffres

In [19]:
df_pdv['len_cp_tmp'] = df_pdv['code_postal'].map(len)

In [20]:
df_pdv['len_cp_tmp'].value_counts()

5    22990
4     1707
Name: len_cp_tmp, dtype: int64

In [21]:
def correct_cp(x):
    if len(x)==4:
        return '0'+x
    return x

In [22]:
df_pdv['code_postal'] = df_pdv['code_postal'].apply(correct_cp)

In [23]:
df_pdv.drop(['len_cp_tmp'], axis=1, inplace=True)

### Activités et environnements

In [24]:
cols = [col for col in df_pdv.columns if 'env' in col or 'acti' in col]

for col in cols:
    df_pdv[col] = df_pdv[col].astype('float')

In [25]:
#df_pdv[cols].isnull().sum()

In [26]:
print(df_pdv[cols].isnull().sum().iloc[0], "lignes ont aucune données ni pour activité ni pour environnement.")
print("Une nouvelle variable booléenne acti_env_inconnu va être créée.")

1118 lignes ont aucune données ni pour activité ni pour environnement.
Une nouvelle variable booléenne acti_env_inconnu va être créée.


In [27]:
df_pdv['activite_env_connu'] = df_pdv.activite_alimentation.notnull().astype(int)
#tous ceux avec pas de valeurs pour act ou env sont mis à 0
for col in cols :
    df_pdv.loc[df_pdv[col].isnull(), col] = 0.0

In [28]:
#for col in cols :
    #print(df_pdv[col].unique())

In [29]:
#df_pdv['activite_env_connu']

### Horaires d'ouverture

#### a) Créer une colonne pour chq jour pour dire si oui ou non c'est ouvert ce jour-là

In [ ]:
import itertools
jours = ['lundi', 'mardi', 'mercredi', 'jeudi', 'vendredi', 'samedi', 'dimanche', 'jourferie']
ouvertures = ['_ouv_am', '_ferm_am', '_ouv_pm', '_ferm_pm']
cols_jours_brut = list(itertools.product(jours, ouvertures))
cols_jours = [col[0] + col[1] for col in cols_jours_brut]
for col in cols_jours :
    df_pdv[col] = df_pdv[col].astype('datetime64[ns]')

In [ ]:
import datetime
def is_open(x, jour):
    for o in ouvertures :
        if x[jour+o] is not pd.NaT :
            if x[jour+o].time() != datetime.time(0, 0) :
                return True
    return False

In [ ]:
for jour in jours: 
    df_pdv[jour + "_is_open"] = df_pdv.apply(lambda x : is_open(x, jour), axis = 1)

In [ ]:
#Exemple
jour = 'lundi'
x = df_pdv.loc[172]
o = '_ouv_am'
#print(x['lundi_ouv_am'].time())
is_open(x, jour)
#x[jour+o].time()

In [ ]:
x = df_pdv.loc[172] 
#x.tail(50)

In [ ]:
def convert_to_time(x):
    if x is not pd.NaT:
        return x.time()
    return x

In [ ]:
for col in cols_jours:
    df_pdv[col + "_in_time"]= df_pdv[col].apply(convert_to_time)

##### b) Traiter les ouvertures à 00h01, 1h et 3h du mat (insignifiantes) et mettre en False

In [ ]:
ouvertures_am = ['_ouv_am']
cols_am = [col[0] + col[1] for col in list(itertools.product(jours, ouvertures_am))]
cols_am

In [ ]:
for col in cols_am:
    df_pdv.loc[df_pdv[col+ "_in_time"] < datetime.time(2, 0),col[:-7]+ "_is_open"] = False

In [ ]:
#TODO horaires incohérents (fermeture avant ouverture)

##### c) Si c'est fermé (False) tous les jours, c'est qu'on a pas les données (pour éviter de tout fausser)

In [ ]:
ouvertures_jours = [jour + "_is_open" for jour in jours]
def is_true_data(x):
    for col in ouvertures_jours:
        if x[col]==True:
            return True
    return False

In [ ]:
df_pdv["horaires_dispo"] = df_pdv.apply(lambda x : is_true_data(x), axis = 1)

In [ ]:
df_pdv["horaires_dispo"].value_counts()

##### d) Si le pdv ouvre un jour, calculer nombres d'heures d'ouverture

In [ ]:
from datetime import datetime, date

def compute_opening_hours(jour, x):
    nombre_secondes = 0
    if x[jour + "_is_open"]==True:
        #si valeurs identiques au milieu (null, 0 ou autre) : pas de changement réel
        if ((x[jour + "_ferm_am"]==x[jour + "_ouv_pm"]) | ((x[jour + "_ferm_am"] is pd.NaT) & (x[jour + "_ferm_am"] is pd.NaT))):
            if (x[jour + "_ferm_pm"] is not pd.NaT):
                if (x[jour + "_ouv_am"] is not pd.NaT):
                    nombre_secondes = x[jour + "_ferm_pm"] - x[jour + "_ouv_am"]
        else:
            if (x[jour + "_ouv_am"] is not pd.NaT):
                    if (x[jour + "_ouv_pm"] is not pd.NaT):
                         if (x[jour + "_ferm_am"] is not pd.NaT):
                                 if (x[jour + "_ferm_pm"] is not pd.NaT):
                                        nombre_secondes = x[jour + "_ferm_pm"]-x[jour + "_ouv_pm"] + x[jour + "_ferm_am"]-x[jour + "_ouv_am"]
    #en forme de time_delta pas besoin de conversion
    return nombre_secondes

In [ ]:
for jour in jours: 
    df_pdv[jour + "_total_hours"] = df_pdv.apply(lambda x : compute_opening_hours(jour,x), axis = 1)

In [ ]:
#y = df_pdv.loc[1]
#y = df_pdv.loc[172]
y = df_pdv.loc[22949]
#y.tail(40)

A retenir : 
- Colonne qui indique si on a les données d'ouvertures : **horaires_dispo** : T/F
- Colonnes pour savoir si c'est ouvert chaque jour : comme **mardi_is_open** : T/F
- Colonnes pour connaitre le nombre d'heures d'ouvertures par jour : comme **mardi_total_hours** : un nombre ou 0 

## 2. Traitement de titres.csv

In [ ]:
def replace_(x):
    try : 
        return float(x)
    except :
        return np.nan

### Noms des colonnes

In [ ]:
df_titres.columns=["distributeur","titre","numero_parution","date_MEV",
                   "prix_vente","plus_produit","type","cout_magazine",
                   "cout_plus_produit","cout_conditionnement",
                   "contrainte_production","prevision_vente"]

### Types des données

In [ ]:
df_titres["prevision_vente"]=df_titres["prevision_vente"].astype(float)
df_titres["contrainte_production"]=df_titres["contrainte_production"].astype(float)
df_titres["prix_vente"]=df_titres["prix_vente"].astype(float)
df_titres["plus_produit"]=df_titres["plus_produit"].astype(int)
df_titres["cout_plus_produit"]=df_titres["cout_plus_produit"].apply(replace_)
df_titres["cout_conditionnement"]=df_titres["cout_conditionnement"].astype(float)
df_titres["cout_magazine"]=df_titres["cout_magazine"].astype(float)

### Dates de mise en vente

- Modification des dates de la forme "mercredi 12 janvier 2019" et "2019-01-12 00:00:00" en 2019-01-12

In [ ]:
mois = {
    "janvier":"01",
    "février":"02",
    "mars":"03",
    "avril":"04",
    "mai":"05",
    "juin":"06",
    "juillet":"07",
    "août":"08",
    "septembre":"09",
    "octobre":"10",
    "novembre":"11",
    "décembre":"12"  
}

def preprocessing_date(liste, column_name):
    res=[]
    for date in liste[column_name]:
        # Split la date pour faciliter son traitement
        date_split=date.split(" ")
        if date[0:2]!="20": #type [[mercredi],[12],[janvier],[2019]]
            if len(date_split[1])!= 2:
                date_split[1]='0'+date_split[1]
            date_corrige=date_split[3]+'-'+mois[date_split[2]]+'-'+date_split[1]
            res.append(date_corrige)
        else : #type [[2019-01-12],[00:00:00]]
            res.append(date_split[0])
    return(res)

In [ ]:
df_titres["date_MEV"]=preprocessing_date(df_titres,"date_MEV")

In [ ]:
df_titres.head()

### Traitement des prix 
- Pour ne garder que 2 décimales au maximum

In [ ]:
def preprocessing_prix(liste, column_name):
    res=[]
    for price in liste[column_name]:
       res.append(round(price,2))
    return(res)

In [ ]:
df_titres["prix_vente"]=preprocessing_prix(df_titres,"prix_vente")
df_titres["cout_magazine"]=preprocessing_prix(df_titres,"cout_magazine")
df_titres["cout_plus_produit"]=preprocessing_prix(df_titres,"cout_plus_produit")
df_titres["cout_conditionnement"]=preprocessing_prix(df_titres,"cout_conditionnement")

## 3. Traitement de ventes.csv

In [ ]:
df_ventes.columns=["titre","paru","pdv","fournis","ventes"]

In [ ]:
df_ventes["Fournis"]=df_ventes["Fournis"].astype(float)
df_ventes["Ventes"]=df_ventes["Ventes"].astype(float)

In [ ]:
df_ventes.shape

In [ ]:
df_ventes.head()

## 4. Export des données

In [61]:
#Nom du dossier où on veut les stocker
path = 'CSV-Clean'

df_pdv.to_csv(path + "/pdv_processed.csv")
#df_titres.to_csv(path + "/titres_processed.csv")
#df_ventes.to_csv(path + "/ventes_processed.csv")